# Creating a grid of photometric fluxes

This tutorial shows how to generate a grid of photometric fluxes 
from a given SSP model, consisting of four dimensions:
- Observed redshift
- Dust attenuation
- Filter band
- SSP metallicity
- SSP age

The SSP photometric grid can then be combined with a given
star formation history, e.g. a log-normal SFH, to produce a grid of photometric fluxes
for different SFHs.


In [ ]:
from matplotlib import pyplot as plt
import os
import numpy as np
from time import time
from astropy import units as u

# Import the SSP model
from pst.SSP import PopStar
# Import the dust extinction model
from pst.dust import DustScreen
# Import the observables
from pst.observables import Filter, load_photometric_filters
# Import the chemical evolution model
from pst.models import LogNormalCEM

In [ ]:
%load_ext autoreload
%autoreload 2

## SSP model
We are going to use the PopStar SSP model and the Chabrier 2003 IFM.

In [ ]:
ssp = PopStar(IMF='cha')

To speed the computation up, we are going to decrease the spectra resolution (to 5 AA) and limit the wavelength range (1000-20000 AA).

In [ ]:
ssp.interpolate_sed(np.arange(1000, 2e4, 5.0) * u.angstrom)

### Filters
In this example, we'll use the PANSTARRS broad band filters g, r, i and z. 

In [ ]:
svo_filter_names = ['PAN-STARRS_PS1.g', 'PAN-STARRS_PS1.r', 'PAN-STARRS_PS1.i', 'PAN-STARRS_PS1.z']
filters = load_photometric_filters(svo_filter_names)

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title(f'Broad-band filters')
ax.set_ylabel('filter response')
ax.set_xlabel('wavelength [$\AA$]')

for ff, lbl in zip(filters, svo_filter_names):
    ff.interpolate(ssp.wavelength)
    ax.plot(ff.wavelength, ff.response, label=lbl)

ax.legend()

### Dust extinction
We are also interested on modelling the effects of dust attenuation on the broad band photometry. Let's create a dust screen model based on Cardelli+89 extinction law

In [ ]:
dust_model = DustScreen("ccm89")

### Photometric grid
Finally, we may also want to include the effects of redshift on the observed photometry. We can declare the array of extinctions (in terms of $A_V$) and the range of observed redshifts that are going to use to generate the grid.

In [ ]:
a_v_array = np.linspace(0, 2, 21)
redshift_array = np.linspace(0, 1, 11)
print('A_V =', a_v_array)
print('z =', redshift_array)

Now it is time to produce the grid

In [ ]:
tstart = time()
all_photometry = np.zeros(
    (redshift_array.size, a_v_array.size, len(filters),
                           *ssp.L_lambda.shape[:-1])) << u.Jy/u.Msun

for i, z in enumerate(redshift_array):
    print(f'Computing photometric grid for redshift z={z}')
    for j, av in enumerate(a_v_array):
        # For each value of AV, we create a new SSP model
        red_ssp = dust_model.redden_ssp_model(ssp, a_v=av)
        # Compute the SSP photometry at the observed redhisft z
        all_photometry[i, j] = red_ssp.compute_photometry(filter_list=filters,
                                                          z_obs=z, verbose=False)

tend = time()
print(f"time spent generating SSP photometry: {tend - tstart:.2f} s")
print("time spent generating a single SSP model photometry: ",
      f"{(tend - tstart) / (all_photometry.shape[0] * all_photometry.shape[1]):.2f} s")

Then we end up with a grid that has four dimensions (redshift, Av, filter, metallicity, age)

In [ ]:
all_photometry.shape

Let us explore the relation between age, metallicity, and dust extinction

In [ ]:
fig = plt.figure(figsize=(12, 7))

idx_redshift = 0
fig.suptitle(f'redshift z = {redshift_array[idx_redshift]}')
idx_filter1 = 0
idx_filter2 = 1
for idx_ax, idx_a_v in enumerate([0, 5, 10, 20]):
    ax = fig.add_subplot(2, 4, idx_ax+1)
    ax.set_title(f'A_V = {a_v_array[idx_a_v]}')
    #ax.set_xlabel('age [Gyr]')
    ax.set_xscale('log')
    ax.set_xlim(6e-4, 15)
    if idx_ax == 0:
        ax.set_ylabel('metallicity Z')
    #ax.set_yscale('log')
    ax.set_ylim(5e-5, 0.05)
    colour = -2.5 * np.log10(all_photometry[idx_redshift, idx_a_v, idx_filter1] / all_photometry[idx_redshift, idx_a_v, idx_filter2])
    mappable = ax.pcolormesh(ssp.ages.to_value("Gyr"), ssp.metallicities, colour,
                            vmin=-.3, vmax=1.1, cmap='rainbow')
plt.colorbar(mappable, ax=ax, label=f"{svo_filter_names[idx_filter1]} - {svo_filter_names[idx_filter2]}")

for idx_ax, idx_met in enumerate([2, 3, 4, 5]):
    ax = fig.add_subplot(2, 4, idx_ax+5)
    ax.set_title(f'Z = {ssp.metallicities[idx_met]}')
    ax.set_xlabel('age [Gyr]')
    ax.set_xscale('log')
    ax.set_xlim(6e-4, 15)
    if idx_ax == 0:
        ax.set_ylabel('A_V')
    #ax.set_yscale('log')
    #ax.set_ylim(5e-5, 0.05)
    colour = -2.5 * np.log10(all_photometry[idx_redshift, :, idx_filter1, idx_met] / all_photometry[idx_redshift, :, idx_filter2, idx_met])
    mappable = ax.pcolormesh(ssp.ages.to_value("Gyr"), a_v_array, colour,
                            vmin=-.3, vmax=1.1, cmap='rainbow')
plt.colorbar(mappable, ax=ax, label=f"{svo_filter_names[idx_filter1]} - {svo_filter_names[idx_filter2]}")


Repeating this experiment at $z = 0.1$ illustrates the importance of an accurate k-correction

In [ ]:
fig = plt.figure(figsize=(12, 7))

idx_redshift = 2
fig.suptitle(f'redshift z = {redshift_array[idx_redshift]}')
idx_filter1 = 0
idx_filter2 = 1
for idx_ax, idx_a_v in enumerate([0, 5, 10, 20]):
    ax = fig.add_subplot(2, 4, idx_ax+1)
    ax.set_title(f'A_V = {a_v_array[idx_a_v]}')
    #ax.set_xlabel('age [Gyr]')
    ax.set_xscale('log')
    ax.set_xlim(6e-4, 15)
    if idx_ax == 0:
        ax.set_ylabel('metallicity Z')
    ax.set_yscale('log')
    ax.set_ylim(5e-5, 0.05)
    colour = -2.5 * np.log10(all_photometry[idx_redshift, idx_a_v, idx_filter1] / all_photometry[idx_redshift, idx_a_v, idx_filter2])
    mappable = ax.pcolormesh(ssp.ages.to_value("Gyr"), ssp.metallicities, colour,
                            vmin=-.3, vmax=1.1, cmap='rainbow')
plt.colorbar(mappable, ax=ax, label=f"{svo_filter_names[idx_filter1]} - {svo_filter_names[idx_filter2]}")

for idx_ax, idx_met in enumerate([2, 3, 4, 5]):
    ax = fig.add_subplot(2, 4, idx_ax+5)
    ax.set_title(f'Z = {ssp.metallicities[idx_met]}')
    ax.set_xlabel('age [Gyr]')
    ax.set_xscale('log')
    ax.set_xlim(6e-4, 15)
    if idx_ax == 0:
        ax.set_ylabel('A_V')
    #ax.set_yscale('log')
    #ax.set_ylim(5e-5, 0.05)
    colour = -2.5 * np.log10(all_photometry[idx_redshift, :, idx_filter1, idx_met] / all_photometry[idx_redshift, :, idx_filter2, idx_met])
    mappable = ax.pcolormesh(ssp.ages.to_value("Gyr"), a_v_array, colour,
                            vmin=-.3, vmax=1.1, cmap='rainbow')
plt.colorbar(mappable, ax=ax, label=f"{svo_filter_names[idx_filter1]} - {svo_filter_names[idx_filter2]}")


## Chemical Evolution Model

Now we can increase the complexity and combine the SSP photometric fluxes according to some chemical evolution model. In this example, we are going to use a log-normal star formation history with a metallicity evolution proportional to the stellar mass: $Z(t)=Z_{today}\, \left(M(t) / M_{today}\right)^\alpha$

In [ ]:
today = 13.7 << u.Gyr
mass_today = 1e11 << u.Msun
ism_metallicity_today = 0.02 << u.dimensionless_unscaled
alpha_powerlaw = 1.5
t0 = 0.1 << u.Gyr
scale = 2.0

In [ ]:
model = LogNormalCEM(
    today=today,
    mass_today=mass_today, t0=t0, scale=scale,
    ism_metallicity_today=ism_metallicity_today, alpha_powerlaw=alpha_powerlaw,
)

Since we are interested on covering a wide range of possible SFHs, we will generate another grid in terms of the free parameters of the SFH

In [ ]:
#t0_array = today - np.geomspace(100 * u.Myr, today - 1*u.Gyr, 10)
t0_array = np.linspace(1, 20, 20) << u.Gyr
scale_array = np.geomspace(0.1, 100, 30)
print('t0 =', t0_array)
print('scale =', scale_array)

all_models_photometry = np.zeros(
    (t0_array.size, scale_array.size, *all_photometry.shape[:-2]))

In [ ]:
tstart = time()
for i, t0 in enumerate(t0_array):
    for j, scale in enumerate(scale_array):
        model.t0 = t0
        model.scale = scale

        all_models_photometry[i, j] = model.compute_photometry(
            ssp, t_obs=ssp.ages.max(), photometry=all_photometry)

tend = time()
print("time spent generating Model photometry: ", tend - tstart)


We now have a grid with five dimensions (t0, scale, redshift, Av, filter)

In [ ]:
all_models_photometry.shape

Let us explore how colour depends on the model parameters t0 and scale

In [ ]:
fig = plt.figure()

idx_redshift = 0
idx_a_v = 0

ax = fig.add_subplot(111)
ax.set_title(f'LogNormalCEM, $z={redshift_array[idx_redshift]}$, $A_V={a_v_array[idx_a_v]}$')
ax.set_ylabel('$t_0$ [Gyr]')
ax.set_xlabel('scale')
ax.set_xscale('log')
colour = -2.5 * np.log10(all_models_photometry[:, :, idx_redshift, idx_a_v, idx_filter1] / all_models_photometry[:, :, idx_redshift, idx_a_v, idx_filter2])
mappable = ax.pcolormesh(scale_array, t0_array.to_value(u.Gyr), colour, vmin=-.3, vmax=1.1, cmap='rainbow')

plt.colorbar(mappable, ax=ax, label=f"{svo_filter_names[idx_filter1]} - {svo_filter_names[idx_filter2]}")
